In [1]:
import numpy as np
import pyprind
import seaborn as sns
import pandas as pd
import os
import pprint
import copy
import wurst as w
import brightway2 as bw
from brightway2 import *
import bw2io as bi
from matplotlib import pyplot as plt
from pandas import DataFrame
from openpyxl import load_workbook
from wurst import *
from wurst.searching import *
import json
from wurst.ecoinvent.electricity_markets import *
from wurst.ecoinvent.filters import *

from pprint import pformat
import warnings
from datetime import datetime

from bw2io.strategies import (
    normalize_units,    
    remove_zero_amount_coproducts,
    normalize_biosphere_categories,
    normalize_biosphere_names,
    strip_biosphere_exc_locations,
)

import collections

In [2]:
projects.set_current("elec_mix_version3")
bw.bw2setup() 

Biosphere database already present!!! No setup is needed


In [3]:
carma_ccs_db = 'Carma CCS'
if carma_ccs_db in databases:
    print("Deleting Carma CCS database")
    del databases['Carma CCS']
else:
    print("No Carma CCS to delete")

Deleting Carma CCS database


In [4]:
wave_db = 'Wave'
if wave_db in databases:
    print("Deleting Wave databse...")
    del databases['Wave']
else:
    print("No Wave to delete")

Deleting Wave databse...


In [5]:
ecoi_db = 'ecoinvent3.6_RMP'
if ecoi_db in databases:
    print("Deleting ecoinvent3.6_RMP databse...")
    del databases['ecoinvent3.6_RMP']
else:
    print("No ecoinvent3.6_RMP to delete")

Deleting ecoinvent3.6_RMP databse...
Vacuuming database 


In [6]:
ei36_path_test = "/Users/sanjanaryali/rmp/ecoinvent/datasets"
ei_36 = SingleOutputEcospold2Importer(ei36_path_test, "ecoinvent3.6_RMP")
ei_36.apply_strategies()
ei_36.statistics()
ei_36.write_database()

Extracting XML data from 18121 datasets
Extracted 18121 datasets in 32.39 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: 

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:27


Title: Writing activities to SQLite3 database:
  Started: 07/17/2021 12:44:18
  Finished: 07/17/2021 12:44:45
  Total time elapsed: 00:00:27
  CPU %: 2.40
  Memory %: 79.92
Created database: ecoinvent3.6_RMP


Brightway2 SQLiteBackend: ecoinvent3.6_RMP

In [7]:
lci_wave_path = "/Users/sanjanaryali/rmp/ecoinvent/data/lci_wave.xlsx"
sp = ExcelImporter(lci_wave_path)
sp.apply_strategies()  
sp.match_database(fields=["name", "unit", "location"])
sp.match_database('ecoinvent3.6_RMP', fields=["reference product", "name", "unit", "location"])
sp.match_database('ecoinvent3.6_RMP', fields=["name", "unit", "location"])
sp.write_database()

/opt/miniconda3/envs/py39/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/opt/miniconda3/envs/py39/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Extracted 1 worksheets in 0.07 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.11 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/17/2021 12:45:40
  Finished: 07/17/2021 12:45:40
  Total time elapsed: 00:00:00
  CPU %: 2.50
  Memory %: 85.86
Created database: Wave


In [8]:
lci_ccs_path = "/Users/sanjanaryali/rmp/ecoinvent/data/lci_ccs.xlsx"
sw = ExcelImporter(lci_ccs_path)
sw.apply_strategies()  
sw.match_database(fields=["name", "unit", "location"])
sw.match_database('ecoinvent3.6_RMP', fields=["reference product", "name", "unit", "location"])
sw.match_database('ecoinvent3.6_RMP', fields=["name", "unit", "location"])
sw.write_database()

Extracted 1 worksheets in 14.06 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.17 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/17/2021 12:46:01
  Finished: 07/17/2021 12:46:01
  Total time elapsed: 00:00:00
  CPU %: 2.50
  Memory %: 86.29
Created database: Carma CCS


In [9]:
databases

Databases dictionary with 5 object(s):
	Carma CCS
	Wave
	biosphere3
	ecoinvent3.6_RMP
	extended_ecoinvent

In [10]:
input_db = extract_brightway2_databases(['ecoinvent3.6_RMP', 'Wave', 'Carma CCS'])

Getting activity data


  1%|          | 4972/618548 [00:00<00:12, 49714.95it/s]

Adding exchange data to activities


100%|██████████| 618548/618548 [00:29<00:00, 21087.52it/s]


Filling out exchange data


100%|██████████| 18268/18268 [00:01<00:00, 10125.29it/s]


In [11]:
carma_electricity_ds_name_dict = {
    "Electricity, at wood burning power plant 20 MW, truck 25km, post, pipeline 200km, storage 1000m/2025": "Biomass CCS",
    "Electricity, at power plant/natural gas, NGCC, no CCS/2025/kWh": "Natural gas CC",
    "Electricity, at power plant/natural gas, pre, pipeline 400km, storage 3000m/2025": "Natural gas CCS",
    "Electricity, at BIGCC power plant 450MW, pre, pipeline 200km, storage 1000m/2025": "Biomass CCS",
    "Electricity, at power plant/hard coal, PC, no CCS/2025": "Coal ST",
    "Electricity, at power plant/hard coal, IGCC, no CCS/2025": "IGCC",
    "Electricity, at wood burning power plant 20 MW, truck 25km, no CCS/2025": "Biomass ST",
    "Electricity, at power plant/natural gas, pre, pipeline 200km, storage 1000m/2025": "Natural gas CCS",
    "Electricity, at power plant/lignite, PC, no CCS/2025": "Coal ST",
    "Electricity, at power plant/hard coal, pre, pipeline 200km, storage 1000m/2025": "Coal CCS",
    "Electricity, from CC plant, 100% SNG, truck 25km, post, pipeline 200km, storage 1000m/2025": "Biomass CCS",
    "Electricity, at wood burning power plant 20 MW, truck 25km, post, pipeline 400km, storage 3000m/2025": "Biomass CCS",
    "Electricity, at power plant/hard coal, oxy, pipeline 400km, storage 3000m/2025": "Coal CCS",
    "Electricity, at power plant/lignite, oxy, pipeline 200km, storage 1000m/2025": "Coal CCS",
    "Electricity, at power plant/hard coal, post, pipeline 400km, storage 3000m/2025": "Coal CCS",
    "Electricity, at power plant/lignite, pre, pipeline 200km, storage 1000m/2025": "Coal CCS",
    "Electricity, at BIGCC power plant 450MW, pre, pipeline 400km, storage 3000m/2025": "Biomass CCS",
    "Electricity, at power plant/natural gas, post, pipeline 400km, storage 1000m/2025": "Natural gas CCS",
    "Electricity, at power plant/lignite, post, pipeline 400km, storage 3000m/2025": "Coal CCS",
    "Electricity, at power plant/hard coal, post, pipeline 400km, storage 1000m/2025": "Coal CCS",
    "Electricity, from CC plant, 100% SNG, truck 25km, post, pipeline 400km, storage 3000m/2025": "Biomass CCS",
    "Electricity, at power plant/natural gas, ATR H2-CC, no CCS/2025": "Natural gas CCS",
    "Electricity, at power plant/hard coal, pre, pipeline 400km, storage 3000m/2025": "Coal CCS",
    "Electricity, at power plant/lignite, IGCC, no CCS/2025": "IGCC",
    "Electricity, at power plant/hard coal, post, pipeline 200km, storage 1000m/2025": "Coal CCS",
    "Electricity, at power plant/lignite, oxy, pipeline 400km, storage 3000m/2025": "Coal CCS",
    "Electricity, at power plant/lignite, post, pipeline 200km, storage 1000m/2025": "Coal CCS",
    "Electricity, at power plant/lignite, pre, pipeline 400km, storage 3000m/2025": "Coal CCS",
    "Electricity, at power plant/natural gas, post, pipeline 200km, storage 1000m/2025": "Natural gas CCS",
    "Electricity, at power plant/natural gas, post, pipeline 400km, storage 3000m/2025": "Natural gas CCS",
    "Electricity, at BIGCC power plant 450MW, no CCS/2025": "Biomass ST",
    "Electricity, from CC plant, 100% SNG, truck 25km, no CCS/2025": "Biomass ST",
    "Electricity, at power plant/hard coal, oxy, pipeline 200km, storage 1000m/2025": "Coal CCS",
}

In [12]:
def fix_unset_technosphere_and_production_exchange_locations(
    db, matching_fields=("name", "unit")
):
    """Fills in all missing location fields for technosphere and production exchanges.

    For production exchanges the location is set to the location of the activity which the exchange is part of.
    For technosphere exchanges, loactions are instead set to the location of another activity
    matching the exchange. If there is not exactly one such activity, the location will not be changed.

    Parameters
    ----------
    db
        Database to fix locations for. Expected to be in wurst format.
    matching_fields: iterable
        Names of the fields that technosphere exchanges and activities in the database are matched on.
    """
    for ds in db:
        for exc in ds["exchanges"]:
            if exc["type"] == "production" and exc.get("location") is None:
                exc["location"] = ds["location"]
            elif exc["type"] == "technosphere" and exc.get("location") is None:
                locs = _find_location_given_lookup_dict(
                    db, {k: exc.get(k) for k in matching_fields}
                )
                if len(locs) == 1:
                    exc["location"] = locs[0]
                else:
                    print(
                        "No unique location found for exchange:\n{}\nFound: {}".format(
                            pprint.pformat(exc), locs
                        )
                    )

In [13]:
def _find_location_given_lookup_dict(db, lookup_dict):
    return [
        x["location"]
        for x in get_many(db, *[equals(k, v) for k, v in lookup_dict.items()])
    ]


In [14]:
def set_global_location_for_additional_datasets(db, ei_name):
    """Set the location of all added datasets to the global location (``GLO``).

    For this function, we assume that all datasets, whose database code is not ``ei_name``, are externally added.
    All added datasets and all exchanges of the same name will have their location changed to ``GLO`` .

    This function is needed because the wurst function relink_technosphere exchanges needs global datasets if it cannot find a regional one.


    Parameters
    ----------
    db
        Database to change locations on. Expected to be in wurst format.
    ei_name : str
        Name of the ecoinvent database. Activities from this database will not have their locations changed.
    """
    non_ecoinvent_datasets = [x["name"] for x in db if x["database"] != ei_name]
    ecoinvent_datasets = [x["name"] for x in db if x["database"] == ei_name]
    print(
        "Found {} ecoinvent datasets and {} non-ecoinvent datasets.".format(
            len(ecoinvent_datasets), len(non_ecoinvent_datasets)
        )
    )

    for ds in [x for x in db if x["database"] != ei_name]:
        ds["location"] = "GLO"
        for exc in [x for x in ds["exchanges"] if x["type"] != "biosphere"]:
            if exc["name"] in non_ecoinvent_datasets:
                if exc["name"] in ecoinvent_datasets and exc["location"] != "GLO":
                    print(exc["name"], exc["location"])
                    print("here")
                else:
                    exc["location"] = "GLO"
                    # print("Set location to 'GLO' for exchange: {}".format(exc['name']))

In [15]:
exists = lambda x: {k: v for k, v in x.items() if v is not None}
def remove_nones(db):
    """Removes all fields of all exchanges whose value is ``None``.

    Parameters
    ----------
    db
        Database to remove empty exchange fields for. Expected to be in wurst format.
    """
    for ds in db:
        ds["exchanges"] = [exists(exc) for exc in ds["exchanges"]]
        


In [16]:
def add_new_locations_to_added_datasets(db):
    """Creates copies of added electricity activities for IEA locations.

    For each electricity activity, only one localised activity will be created.


    Parameters
    ----------
    db
        Database to add localised activities to. Expected to be in wurst format.


    See Also
    --------
    regionalize_added_datasets : Regionalize exchanges added using this function.
    """
    # We create a new version of all added electricity generation datasets for each IEA region.
    # We allow the upstream production to remain global, as we are mostly interested in regionalizing
    # to take advantage of the regionalized IEA data.

    # step 1: make copies of all datasets for new locations
    # best would be to regionalize datasets for every location with an electricity market like this:
    # locations = {x['location'] for x in get_many(db, *electricity_market_filter_high_voltage)}
    # but this takes quite a long time. For now, we just use 1 location that is unique in each IEA region.
    possibles = {}
#     for reg in REGIONS_IEA[:-1]:
#         print(reg)
#         temp = [x for x in geomatcher.intersects(("IEA", reg)) if type(x) != tuple]
#         possibles[reg] = [x for x in temp if len(ecoinvent_to_iea_locations(x)) == 1]
#         if not len(possibles[reg]):
#             print(reg, " has no good candidate")
    locations = [v[0] for v in possibles.values()]
    print(locations)

    # This code would modify every new dataset, but this would be quite large:
    # for ds in  pyprind.prog_bar([ds for ds in db if ds['database'] in ['CSP','Carma CCS']]):
    # so we consider only the final electricity production dataset and not the upstream impacts:
    for ds in pyprind.prog_bar(
        [ds for ds in db if ds["name"] in carma_electricity_ds_name_dict.keys()]
    ):
        for location in locations:
            new_ds = copy_to_new_location(ds, location)
            db.append(new_ds)




In [17]:
def regionalize_added_datasets(db):
    """Links exchanges of added electricity activities to regionalized activities.

    Parameters
    ----------
    db
        Database to regionalize activities for. Expected to be in wurst format.
    """
    # step 2: relink all processes in each dataset
    # This code would modify every new dataset, but this would be quite large:
    # for ds in  pyprind.prog_bar([ds for ds in db if ds['database'] in ['CSP','Carma CCS']]):
    # so we consider only the final electricity production dataset and not the upstream impacts:
    for ds in [ds for ds in db if ds["name"] in carma_electricity_ds_name_dict.keys()]:
        ds = relink_technosphere_exchanges(
            ds,
            db,
            exclusive=True,
            drop_invalid=False,
            biggest_first=False,
            contained=False,
        )


In [18]:
## Set the location for activities that does not have locations
w.transformations.default_global_location(input_db)
## Set the location (as GLO) for eschanges (technosphere and production) that does not have locations
fix_unset_technosphere_and_production_exchange_locations(input_db)

set_global_location_for_additional_datasets(input_db, 'ecoinvent3.6_RMP')
remove_nones(input_db)

add_new_locations_to_added_datasets(input_db)

regionalize_added_datasets(input_db)

Found 18121 ecoinvent datasets and 147 non-ecoinvent datasets.
[]


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [20]:
extended_ecoinvent_db = 'extended_ecoinvent'
if carma_ccs_db in databases:
    print("Deleting extended_ecoinvent")
    del databases['extended_ecoinvent']
else:
    print("No extended_ecoinvent to delete")

Deleting extended_ecoinvent
Vacuuming database 


In [21]:
db = input_db
write_brightway2_database(db, 'extended_ecoinvent')

18268 datasets
618548 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:26


Title: Writing activities to SQLite3 database:
  Started: 07/17/2021 12:48:29
  Finished: 07/17/2021 12:48:56
  Total time elapsed: 00:00:26
  CPU %: 2.40
  Memory %: 184.96
Created database: extended_ecoinvent


In [22]:
databases

Databases dictionary with 5 object(s):
	Carma CCS
	Wave
	biosphere3
	ecoinvent3.6_RMP
	extended_ecoinvent

In [23]:
df = pd.read_excel ("/Users/sanjanaryali/rmp/ecoinvent/data/elec_mix.xlsx") 

In [24]:
ee = bw.Database('extended_ecoinvent')

In [25]:
%run -i 'ee_activities_DAC.py'

In [26]:
# SAVING ACTIVITIES: DO NOT RUN AGAIN!

ac_construction.save()
pellet_reactor_construction.save()
calciner_construction.save()
other_equipment.save()
dac_operation_material.save()
dac_operation_heat.save()
dac_operation_water.save()
dac_eol.save()


In [27]:
# SET FUNCTIONAL UNIT

air_contactor_construction_fu = {ac_construction:1}
pellet_reactor_construction_fu = {pellet_reactor_construction:1}
calciner_construction_fu = {calciner_construction:1}
other_equipment_fu = {other_equipment:1}
dac_operation_material_fu = {dac_operation_material:1}
dac_operation_heat_fu = {dac_operation_heat:1}
dac_operation_water_fu = {dac_operation_water:1}
dac_eol_fu = {dac_eol:1}

In [28]:
# SAVE RESULTS IN LIST
air_contactor_construction_result = []
pellet_reactor_construction_result = []
calciner_construction_result = []
other_equipment_result = []
dac_operation_material_result = []
dac_operation_heat_result = []
dac_operation_water_result = []
dac_eol_result = []

In [29]:
# CREATE DICT

DAC_dict = collections.defaultdict(dict)

DAC_dict['ac_construction'] = {'fu' : air_contactor_construction_fu, 'result': air_contactor_construction_result}
DAC_dict['pellet_reactor_construction'] = {'fu' : pellet_reactor_construction_fu, 'result': pellet_reactor_construction_result}
DAC_dict['calciner_construction'] = {'fu' : calciner_construction_fu, 'result': calciner_construction_result}
DAC_dict['other_equipment'] = {'fu' : other_equipment_fu, 'result': other_equipment_result}
DAC_dict['dac_operation_material'] = {'fu' : dac_operation_material_fu, 'result': dac_operation_material_result}
DAC_dict['dac_eol'] = {'fu' : dac_eol_fu, 'result': dac_eol_result}
DAC_dict['dac_operation_heat'] = {'fu' : dac_operation_heat_fu, 'result': dac_operation_heat_result}
DAC_dict['dac_operation_water'] = {'fu' : dac_operation_water_fu, 'result': dac_operation_water_result}


In [30]:
#DEFINE LCIA METHOD

method_key = [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'climate change' in str(m)][0],

In [31]:
#CALCULATE LCIA

for key in DAC_dict.keys():
    lca = bw.LCA(DAC_dict[key]['fu'])
    lca.lci()
    for method in method_key:
        lca.switch_method(method)
        lca.lcia()
        DAC_dict[key]['result'].append((method[2].title(), lca.score, bw.methods.get(method).get('unit')))

In [49]:
# SUBTRACTING COMBUSTION FROM NATURAL GAS VALUE (TUPLE TO LIST)
new_list = list(DAC_dict['dac_operation_heat']['result'][0])
print(new_list)
new_list[1] = new_list[1] - 314
print(new_list)
DAC_dict['dac_operation_heat']['result'][0] = tuple(new_list)

['Gwp100', 429.28613821439626, 'kg CO2-Eq']
['Gwp100', 115.28613821439626, 'kg CO2-Eq']


In [50]:
DAC_dict

defaultdict(dict,
            {'ac_construction': {'fu': {'Air Contactor Construction' (unit, GLO, None): 1},
              'result': [('Gwp100', 4.26303737951869, 'kg CO2-Eq')]},
             'pellet_reactor_construction': {'fu': {'Pellet Reactor Construction' (unit, GLO, None): 1},
              'result': [('Gwp100', 1.035889629293701, 'kg CO2-Eq')]},
             'calciner_construction': {'fu': {'Calciner Construction' (unit, GLO, None): 1},
              'result': [('Gwp100', 0.9070065242592056, 'kg CO2-Eq')]},
             'other_equipment': {'fu': {'Other Equipment' (unit, GLO, None): 1},
              'result': [('Gwp100', 1.784814259889664, 'kg CO2-Eq')]},
             'dac_operation_material': {'fu': {'DAC Operation Material' (kg, GLO, None): 1},
              'result': [('Gwp100', 9.71631990443574, 'kg CO2-Eq')]},
             'dac_eol': {'fu': {'DAC End of life' (unit, GLO, None): 1},
              'result': [('Gwp100', -8.297249355737755, 'kg CO2-Eq')]},
             'dac_o

In [55]:
total_dac_score_no_elec = 0;

for key in DAC_dict.keys():
    print(DAC_dict[key]['result'][0][1])
    total_dac_score_no_elec = total_dac_score_no_elec + DAC_dict[key]['result'][0][1]
print("total_dac_score_no_elec: "+ str(total_dac_score_no_elec))

4.26303737951869
1.035889629293701
0.9070065242592056
1.784814259889664
9.71631990443574
-8.297249355737755
115.28613821439626
3.6620240278484344
total_dac_score_no_elec: 128.35798058390395


In [56]:
%run -i 'ee_activities_US.py'

In [57]:
us_2020.save()
us_2030.save()
us_2040.save()
us_2050.save()

In [58]:
us_2020_fu = {us_2020:1}
us_2020_result = []

us_2030_fu = {us_2030:1}
us_2030_result = []

us_2040_fu = {us_2040:1}
us_2040_result = []

us_2050_fu = {us_2050:1}
us_2050_result = []

In [59]:
%run -i 'ee_activities_BRAZIL.py'

In [60]:
brazil_2020.save()
brazil_2030.save()
brazil_2040.save()
brazil_2050.save()

In [61]:
brazil_2020_fu = {brazil_2020:1}
brazil_2020_result = []

brazil_2030_fu = {brazil_2030:1}
brazil_2030_result = []

brazil_2040_fu = {brazil_2040:1}
brazil_2040_result = []

brazil_2050_fu = {brazil_2050:1}
brazil_2050_result = []

In [62]:
%run -i 'ee_activities_RUSSIA.py'

In [63]:
russia_2020.save()
russia_2030.save()
russia_2040.save()
russia_2050.save()

In [64]:
russia_2020_fu = {russia_2020:1}
russia_2020_result = []

russia_2030_fu = {russia_2030:1}
russia_2030_result = []

russia_2040_fu = {russia_2040:1}
russia_2040_result = []

russia_2050_fu = {russia_2050:1}
russia_2050_result = []

In [65]:
%run -i 'ee_activities_INDIA.py'

In [66]:
india_2020.save()
india_2030.save()
india_2040.save()
india_2050.save()

In [67]:
india_2020_fu = {india_2020:1}
india_2020_result = []

india_2030_fu = {india_2030:1}
india_2030_result = []

india_2040_fu = {india_2040:1}
india_2040_result = []

india_2050_fu = {india_2050:1}
india_2050_result = []

In [68]:
%run -i 'ee_activities_EUROPE.py'

In [69]:
europe_2020.save()
europe_2030.save()
europe_2040.save()
europe_2050.save()

In [70]:
europe_2020_fu = {europe_2020:1}
europe_2020_result = []

europe_2030_fu = {europe_2030:1}
europe_2030_result = []

europe_2040_fu = {europe_2040:1}
europe_2040_result = []

europe_2050_fu = {europe_2050:1}
europe_2050_result = []

In [71]:
%run -i 'ee_activities_CHINA.py'

In [72]:
china_2020.save()
china_2030.save()
china_2040.save()
china_2050.save()

In [73]:
china_2020_fu = {china_2020:1}
china_2020_result = []

china_2030_fu = {china_2030:1}
china_2030_result = []

china_2040_fu = {china_2040:1}
china_2040_result = []

china_2050_fu = {china_2050:1}
china_2050_result = []

In [74]:
elec_cal_dict = collections.defaultdict(dict)

elec_cal_dict['us_2020'] = {'fu' : us_2020_fu, 'result': us_2020_result}
elec_cal_dict['us_2030'] = {'fu' : us_2030_fu, 'result': us_2030_result}
elec_cal_dict['us_2040'] = {'fu' : us_2040_fu, 'result': us_2040_result}
elec_cal_dict['us_2050'] = {'fu' : us_2050_fu, 'result': us_2050_result}

elec_cal_dict['brazil_2020'] = {'fu' : brazil_2020_fu, 'result': brazil_2020_result}
elec_cal_dict['brazil_2030'] = {'fu' : brazil_2030_fu, 'result': brazil_2030_result}
elec_cal_dict['brazil_2040'] = {'fu' : brazil_2040_fu, 'result': brazil_2040_result}
elec_cal_dict['brazil_2050'] = {'fu' : brazil_2050_fu, 'result': brazil_2050_result}

elec_cal_dict['russia_2020'] = {'fu' : russia_2020_fu, 'result': russia_2020_result}
elec_cal_dict['russia_2030'] = {'fu' : russia_2030_fu, 'result': russia_2030_result}
elec_cal_dict['russia_2040'] = {'fu' : russia_2040_fu, 'result': russia_2040_result}
elec_cal_dict['russia_2050'] = {'fu' : russia_2050_fu, 'result': russia_2050_result}

elec_cal_dict['india_2020'] = {'fu' : india_2020_fu, 'result': india_2020_result}
elec_cal_dict['india_2030'] = {'fu' : india_2030_fu, 'result': india_2030_result}
elec_cal_dict['india_2040'] = {'fu' : india_2040_fu, 'result': india_2040_result}
elec_cal_dict['india_2050'] = {'fu' : india_2050_fu, 'result': india_2050_result}

elec_cal_dict['europe_2020'] = {'fu' : europe_2020_fu, 'result': europe_2020_result}
elec_cal_dict['europe_2030'] = {'fu' : europe_2030_fu, 'result': europe_2030_result}
elec_cal_dict['europe_2040'] = {'fu' : europe_2040_fu, 'result': europe_2040_result}
elec_cal_dict['europe_2050'] = {'fu' : europe_2050_fu, 'result': europe_2050_result}

elec_cal_dict['china_2020'] = {'fu' : china_2020_fu, 'result': china_2020_result}
elec_cal_dict['china_2030'] = {'fu' : china_2030_fu, 'result': china_2030_result}
elec_cal_dict['china_2040'] = {'fu' : china_2040_fu, 'result': china_2040_result}
elec_cal_dict['china_2050'] = {'fu' : china_2050_fu, 'result': china_2050_result}

In [75]:
method_key = [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'climate change' in str(m)][0],

In [76]:
for key in elec_cal_dict.keys():
    lca = bw.LCA(elec_cal_dict[key]['fu'])
    lca.lci()
    for method in method_key:
        lca.switch_method(method)
        lca.lcia()
        total_score = total_dac_score_no_elec + lca.score
        print(key + ": " + str(total_score))
        elec_cal_dict[key]['result'].append((method[2].title(), total_score, bw.methods.get(method).get('unit')))

us_2020: 293.73833453805656
us_2030: 199.21440669389216
us_2040: 155.21180167831952
us_2050: 157.4172497259616
brazil_2020: 166.39419779986122
brazil_2030: 136.87152095251764
brazil_2040: 133.91686954262136
brazil_2050: 141.7070453428733
russia_2020: 300.8081901427695
russia_2030: 284.60013279710154
russia_2040: 193.3840581793392
russia_2050: 161.02120989791422
india_2020: 388.6263593954289
india_2030: 210.63946239656218
india_2040: 153.37336015661958
india_2050: 155.137786702241
europe_2020: 234.2127914340325
europe_2030: 182.2455237461114
europe_2040: 150.8913599663207
europe_2050: 158.92290895214646
china_2020: 362.44094856332663
china_2030: 244.41859670730173
china_2040: 146.3495644912313
china_2050: 148.26856234209828
